In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [7]:
X = scaler.fit_transform(X)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test , Y_train,Y_test = train_test_split(X,Y ,stratify=Y , test_size= 0.2 , random_state=1)

In [50]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [11]:
model = Sequential()

model.add(Dense(8 , activation='relu' , input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

e:\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.fit(X_train,Y_train , batch_size= 32 , epochs=100 , validation_data=(X_test,Y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.3521 - loss: 0.7930 - val_accuracy: 0.3506 - val_loss: 0.7770
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3542 - loss: 0.7598 - val_accuracy: 0.4091 - val_loss: 0.7507
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3657 - loss: 0.7509 - val_accuracy: 0.4221 - val_loss: 0.7298
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3670 - loss: 0.7432 - val_accuracy: 0.4935 - val_loss: 0.7127
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4415 - loss: 0.7250 - val_accuracy: 0.5325 - val_loss: 0.7004
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5083 - loss: 0.7022 - val_accuracy: 0.5714 - val_loss: 0.6895
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5545 - loss: 0.6848 - val_accuracy: 0.6234 - val_loss: 0.6805
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6186 - loss: 0.6767 - val_accuracy: 0.6688 

In [13]:
import keras_tuner as kt

In [14]:
# Finding the best optimizer using the Keras tuner library


def model_build(hp):
    
    model = Sequential()

    model.add(Dense(32,activation='relu',input_dim= 8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer' , values=['Adam','sgd', 'rmsprop','adadelta'])

    model.compile(optimizer=optimizer , loss= 'binary_crossentropy' , metrics=['accuracy'])

    return model

In [15]:
tuner  = kt.RandomSearch(model_build,
                         objective='val_accuracy',
                         max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [16]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [18]:
model_1 = tuner.get_best_models(num_models=1)[0]

e:\python\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
model_1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(X_train,Y_train , epochs=100 , initial_epoch=6,validation_data=(X_test,Y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8133 - loss: 0.4133 - val_accuracy: 0.7338 - val_loss: 0.5385
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7994 - loss: 0.4232 - val_accuracy: 0.7338 - val_loss: 0.5385
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7896 - loss: 0.4347 - val_accuracy: 0.7403 - val_loss: 0.5392
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8096 - loss: 0.4137 - val_accuracy: 0.7403 - val_loss: 0.5398
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.7747 - loss: 0.4622 - val_accuracy: 0.7403 - val_loss: 0.5396
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7975 - loss: 0.4208 - val_accuracy: 0.7338 - val_loss: 0.5404
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8218 - loss: 0.4002 - val_accuracy: 0.7468 - val_loss: 0.5411
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8022 - loss: 0.4334 - val_accurac

In [21]:
# Finding the best Neurons in layer using the Keras tuner library


def build_model(hp):

    model = Sequential()

    units = hp.Int('units' , 8,128,step=8)       # ( initial no of neurons, final no of neurons , steps to change the neurons (int steps not given then take step=1 or random step))

    model.add(Dense(units=units , activation='relu', input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss ='binary_crossentropy' , metrics=['accuracy'])

    return model



In [27]:
# tuner = kt.RandomSearch(build_model,
#                       objective='val_accuracy',
#                        max_trials=5,
#                       # directory='mydir',
                        #project_name='tuning_models'
#                         )


tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    overwrite=True,
    directory="mydir",
    project_name="tuning_models"
)

e:\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
tuner.search(X_train,Y_train , epochs=5,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 14s


In [29]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [30]:
model_2 = tuner.get_best_models(num_models=1)[0]

e:\python\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [31]:
model_2.fit(X_train,Y_train,epochs=100,initial_epoch=6,validation_data=(X_test,Y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8038 - loss: 0.4429 - val_accuracy: 0.7338 - val_loss: 0.5191
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7725 - loss: 0.4778 - val_accuracy: 0.7532 - val_loss: 0.5184
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7811 - loss: 0.4478 - val_accuracy: 0.7403 - val_loss: 0.5188
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7997 - loss: 0.4307 - val_accuracy: 0.7403 - val_loss: 0.5211
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7959 - loss: 0.4466 - val_accuracy: 0.7468 - val_loss: 0.5221
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7980 - loss: 0.4407 - val_accuracy: 0.7403 - val_loss: 0.5251
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8141 - loss: 0.4084 - val_accuracy: 0.7338 - val_loss: 0.5255
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8062 - loss: 0.4114 - val_accuracy: 0.74

In [32]:
# Finding the best **No of hidden layer** using the Keras tuner library


def build_model(hp):

    model = Sequential()

    model.add(Dense(96 , activation='relu' , input_dim=8))

    for i in range(hp.Int('num_layers' , min_value= 1, max_value= 10)):
        model.add(Dense(96, activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop' , loss= 'binary_crossentropy', metrics=['accuracy'])

    return model
        


In [33]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory= 'mydir',
    project_name='layer_tuning'
)

e:\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
tuner.search(X_train,Y_train,epochs=5 , validation_data= (X_test,Y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 18s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [36]:
model_3 = tuner.get_best_models(num_models=1)[0]

e:\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
e:\python\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [37]:
model_3.fit(X_train,Y_train , epochs=100 , initial_epoch=6 , validation_data=(X_test,Y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7955 - loss: 0.4096 - val_accuracy: 0.7208 - val_loss: 0.6059
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8170 - loss: 0.3928 - val_accuracy: 0.7013 - val_loss: 0.5844
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8167 - loss: 0.3956 - val_accuracy: 0.7013 - val_loss: 0.6600
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8112 - loss: 0.3870 - val_accuracy: 0.7273 - val_loss: 0.5762
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8487 - loss: 0.3605 - val_accuracy: 0.7143 - val_loss: 0.6221
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8378 - loss: 0.3718 - val_accuracy: 0.7208 - val_loss: 0.6469
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8392 - loss: 0.3307 - val_accuracy: 0.7403 - val_loss: 0.6070
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8384 - loss: 0.3433 - val_accuracy: 0.72

# Everything in one function

In [44]:
def build_model(hp):

    model = Sequential()

    counter = 0 

    for i in range(hp.Int('num_layer' , min_value= 1 , max_value = 10,)):
        
        if counter == 0:
            model.add(Dense(
                hp.Int('units'+str(i) , min_value=8, max_value=256, step= 8),
                activation= hp.Choice('activaion'+str(i) , values=['relu' , 'sigmoid' , 'tanh']),
                input_dim=8
            ))

        else:
            model.add(Dense(
                hp.Int('units'+str(i) , min_value=8, max_value=256, step= 8),
                activation= hp.Choice('activaion'+str(i) , values=['relu' , 'sigmoid' , 'tanh'])
            ))
        
        counter +=1

    model.add(Dense(1 , activation='sigmoid'))


    model.compile(
            optimizer=hp.Choice('optimizer' , values=['rmsprop' , 'sgd' , 'adam' , 'nadam' , 'adadelta']),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
    
    return model


In [45]:
tuner= kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name = 'Everything_Tuning'
)

In [46]:
tuner.search(X_train,Y_train , epochs=5 , validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 19s]
val_accuracy: 0.350649356842041

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 34s


In [47]:
tuner.get_best_hyperparameters()[0].values

{'num_layer': 3,
 'units0': 32,
 'activaion0': 'relu',
 'optimizer': 'adam',
 'units1': 24,
 'activaion1': 'relu',
 'units2': 192,
 'activaion2': 'sigmoid',
 'units3': 56,
 'activaion3': 'tanh',
 'units4': 144,
 'activaion4': 'sigmoid',
 'units5': 120,
 'activaion5': 'sigmoid'}

In [48]:
model_4 = tuner.get_best_models(num_models=1)[0]

e:\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
e:\python\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [49]:
model_4.fit(X_train,Y_train , epochs= 200 , initial_epoch= 6, validation_data=(X_test, Y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.7691 - loss: 0.4560 - val_accuracy: 0.7468 - val_loss: 0.4957
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7875 - loss: 0.4679 - val_accuracy: 0.7597 - val_loss: 0.4957
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7946 - loss: 0.4559 - val_accuracy: 0.7597 - val_loss: 0.4910
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7872 - loss: 0.4540 - val_accuracy: 0.7532 - val_loss: 0.4956
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7918 - loss: 0.4628 - val_accuracy: 0.7338 - val_loss: 0.4985
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7712 - loss: 0.4687 - val_accuracy: 0.7273 - val_loss: 0.4980
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7949 - loss: 0.4659 - val_accuracy: 0.7338 - val_loss: 0.5022
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8083 - loss: 0.4072 - val_accuracy: 0.7

# With Dropouts

In [51]:
def build_model(hp):

    model = Sequential()

    counter = 0 

    for i in range(hp.Int('num_layer' , min_value= 1 , max_value = 10,)):
        
        if counter == 0:
            model.add(Dense(
                hp.Int('units'+str(i) , min_value=8, max_value=256, step= 8),
                activation= hp.Choice('activaion'+str(i) , values=['relu' , 'sigmoid' , 'tanh']),
                input_dim=8
            ))
            model.add(Dropout(hp.Choice('dropout'+str(i) , values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        else:
            model.add(Dense(
                hp.Int('units'+str(i) , min_value=8, max_value=256, step= 8),
                activation= hp.Choice('activaion'+str(i) , values=['relu' , 'sigmoid' , 'tanh'])
            ))
            model.add(Dropout(hp.Choice('dropout'+str(i) , values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        
        counter +=1

    model.add(Dense(1 , activation='sigmoid'))


    model.compile(
            optimizer=hp.Choice('optimizer' , values=['rmsprop' , 'sgd' , 'adam' , 'nadam' , 'adadelta']),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
    
    return model


In [53]:
tuner= kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name = 'Everything_Tuning_1'
)

e:\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
tuner.search(X_train,Y_train , epochs =5 , validation_data = (X_test,Y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.350649356842041

Best val_accuracy So Far: 0.649350643157959
Total elapsed time: 00h 00m 27s


In [55]:
model_5 = tuner.get_best_models(num_models=1)[0]

e:\python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
e:\python\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [56]:
model_5.fit(X_train,Y_train , epochs=100 , initial_epoch=6 , validation_data=(X_test,Y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.6527 - loss: 0.6765 - val_accuracy: 0.6494 - val_loss: 0.6669
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6267 - loss: 0.6653 - val_accuracy: 0.6494 - val_loss: 0.6504
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6763 - loss: 0.6492 - val_accuracy: 0.6494 - val_loss: 0.6533
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6330 - loss: 0.6629 - val_accuracy: 0.6494 - val_loss: 0.6493
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6273 - loss: 0.6556 - val_accuracy: 0.6494 - val_loss: 0.6483
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6711 - loss: 0.6398 - val_accuracy: 0.6494 - val_loss: 0.6478
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6628 - loss: 0.6413 - val_accuracy: 0.6494 - val_loss: 0.6481
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6370 - loss: 0.6601 - val_accuracy: 0.6